In [ ]:
import pandas as pd
import numpy as np
from tpulse import TinkoffPulse
from time import sleep
from pytz import timezone
from httpx import HTTPStatusError
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import re

import time
from requests.exceptions import ReadTimeout

import warnings

warnings.filterwarnings("ignore")

In [ ]:

def parsing_tpulse(ticker, N, KEYS):

    cursor = None
    raw_data = []

    for _ in range(N):
        try:
            response = pulse.get_posts_by_ticker(ticker, cursor)
            try:
                cursor = response["nextCursor"]
            except:
                print(
                    f"Страниц с постами акции {ticker} меньше {N} - были выгружены все имеющиеся"
                )
                break
            posts = response["items"]
            for post in posts:
                data = {key: post[key] for key in KEYS}
                data["ticker"] = ticker
                data["text"] = post["content"]["text"]
                data["reactionCount"] = post["reactions"]["totalCount"]
                data["reactions_counters"] = post["reactions"]["counters"]
                raw_data.append(data)

        except HTTPStatusError:
            pass
        sleep(0.25)

    result_df = pd.DataFrame(raw_data)
    result_df["inserted"] = pd.to_datetime(result_df["inserted"]).dt.date

    return result_df


pulse = TinkoffPulse()

keys = ["id", "inserted", "commentsCount"]
share_list = [
    "SBER", "GAZP", "T", "X5", "PIKK", "PLZL", "LKOH", "GMKN", "VTBR", "NVTK",
    "YDEX", "TATN", "SMLT", "SPBE", "ROSN", "AFLT", "IRAO", "VKCO", "CHMF",
    "NLMK", "MAGN", "MGNT", "MTLR", "AFKS", "MOEX", "POSI", "ALRS", "BELU",
    "MTSS", "SVCB", "SNGS", "HEAD", "CNRU", "PHOR", "RTKM", "SIBN", "UGLD",
    "RUAL", "HYDR", "BSPB", "FEES", "EUTR", "RNFT", "MRKC", "MDMG", "FLOT",
    "UPRO", "ASTR", "IVAT", "LENT", "WUSH", "UWGN", "ENPG", "TRMK", "MVID",
    "RASP", "RAGR", "MRKP", "SVAV", "AQUA", "RENI", "SOFL", "SGZH", "SFIN",
    "OZPH", "NMTP", "FIXR", "OGKB", "SELG", "LEAS", "CBOM", "VSMO", "IRKT",
    "VSEH", "AKRN", "LSRG", "RTKMP", "SBERP", "SNGSP", "TATNP", "TTLK"
]
result_df = pd.DataFrame(
    columns=[
        "id",
        "inserted",
        "commentsCount",
        "text",
        "reactionCount",
        "reactions_counters",
    ]
)

max_retries, retry_delay, page_to_read, page_to_read_global = 20, 10, 20000, 20000

for share in share_list:
    page_to_read = page_to_read_global
    for attempt in range(1, max_retries + 1):
        try:
            result = parsing_tpulse(share, page_to_read, keys)
        except Exception as e:
            print(
                f"""
При работе с акцией {share} была получена ошибка: {e}.Попытка {attempt} из {max_retries}: таймаут. Ожидаем {retry_delay} секунд перед повтором. Сократим количество страниц по данной акции до {page_to_read // 2}
                """
            )
            page_to_read = round(page_to_read / 1.5)
            time.sleep(retry_delay)
        else:
            print(
                f"Данные по акции {share} выгружены в размере {result.shape} начиная с {min(result['inserted'])}\n"
            )
            result_df = pd.concat([result_df, result], axis=0)
            break
    else:
        print(
            "Превышено максимальное число попыток. Проверьте сеть и доступность сервера."
        )

result_df
